<a href="https://colab.research.google.com/github/graviraja/100-Days-of-NLP/blob/applications%2Fquestion-answering/applications/question-answering/Boolean%20Question%20Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nlp -q
!pip install transformers -q

     |████████████████████████████████| 1.7MB 2.7MB/s 
     |████████████████████████████████| 17.3MB 196kB/s 
     |████████████████████████████████| 245kB 35.1MB/s 
     |████████████████████████████████| 778kB 2.9MB/s 
     |████████████████████████████████| 3.0MB 17.5MB/s 
     |████████████████████████████████| 890kB 31.5MB/s 
     |████████████████████████████████| 1.1MB 41.4MB/s 


In [11]:
import nlp
import time
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [3]:
metric = nlp.load_metric("glue", "sst2")
dataset = nlp.load_dataset("boolq")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Using custom data configuration default


Dataset boolq downloaded and prepared to /root/.cache/huggingface/datasets/boolq/default/0.1.0/a6e32c881d5d801c46b89178e65e4ce34db89aa08d1d98b8732aa6050e8d9243. Subsequent calls will reuse this data.


In [4]:
def encode_dataset(example):
    return tokenizer(example['question'], example['passage'], truncation=True)    

In [5]:
def encode_label(example):
    return {'labels': int(example['answer'])}

In [6]:
dataset = dataset.map(encode_dataset)
dataset = dataset.map(encode_label)
dataset.set_format(columns=['attention_mask', 'input_ids', 'token_type_ids', 'labels'])

In [12]:
def train_and_evaluate(model, optimizer, train_data, valid_data, device='cuda'):
    model.to(device).train()
    for batch in train_data:
        batch.to(device)
        model(**batch)[0].backward()
        optimizer.step()
        optimizer.zero_grad()
    model.eval()
    with torch.no_grad():
        for batch in valid_data:
            batch.to(device)
            predictions = model(**batch)[1].argmax(dim=-1)
            metric.add_batch(predictions=predictions.detach().cpu().numpy(), references=batch['labels'].squeeze(0).detach().cpu().numpy())
    return metric.compute()

In [8]:
def pad(examples):
    return tokenizer.pad(examples, padding='longest', return_tensors='pt')

In [9]:
train_data = DataLoader(dataset['train'], shuffle=True, collate_fn=pad, batch_size=8)
valid_data = DataLoader(dataset['validation'], shuffle=False, collate_fn=pad, batch_size=16)

In [10]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [14]:
for epoch in range(3):
    start_time = time.time()
    eval_metrics = train_and_evaluate(model, optimizer, train_data, valid_data)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f"Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s")
    print(f"\tAccuracy: {eval_metrics['accuracy']:.4f}")

Epoch: 01 | Time: 16m 31.00s
	Accuracy: 0.6492
Epoch: 02 | Time: 16m 24.00s
	Accuracy: 0.7037
Epoch: 03 | Time: 16m 14.00s
	Accuracy: 0.7223
